In [13]:
! pip install numpy==1.26.0 rasterio==1.3.8 shapely==2.0.1 geopandas==0.14.0 torch==2.0.1 torchvision==0.15.2 git+https://github.com/PyTorchLightning/pytorch-lightning

  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-rehbwyvm
  Running command git clone --filter=blob:none --quiet https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-rehbwyvm
  Resolved https://github.com/PyTorchLightning/pytorch-lightning to commit d1f8b0f7669aa808df5fcc2934455be3c83b4bae
  Running command git submodule update --init --recursive -q
  Encountered 31 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .notebooks/course_UvA-DL/06-graph-neural-networks.ipynb
        .notebooks/course_UvA-DL/07-deep-energy-based-generative-models.ipynb
        .notebooks/course_UvA-DL/08-

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# the folder in Google Drive that contains:
## 'test' folder with test rasters
## 'filter_dem' containing the ArcticDEM preprocessed rasters
## the saved model checkpoint with the best dice_coeff
## 'predictions' a folder where the prediced rasters will be saved
BASE_FOLDER = '/content/drive/MyDrive/Research/SIGSPATIAL Competition/data'

# the same folder is available here:
# https://drive.google.com/drive/folders/1cGyDLLmZafQGetRfnqp8NPGrozed6nx-?usp=sharing
# you can copy it and update the path to where it is copied in your directory

In [16]:
! rm -rf ./test_tiles

In [17]:
import os
import rasterio
import numpy as np
import pandas as pd
import torch
import sys
from glob import glob
#import torchvision

output_dir = '.'
print("output directory: "+ output_dir)
os.makedirs(output_dir+'/test_tiles')


DIM = 256
Step = 1

csv_f = open(output_dir + '/test_tile_info.csv', 'w')
csv_f.write('region_id\traster_id\tx\ty\n')
# index = 0
for region_id in range(1,7):
    print(region_id)
    raster_paths = glob(BASE_FOLDER + '/test/region_0%d/*.tif' % region_id)
    dem_paths = BASE_FOLDER + '/filter_dem/dem_region_0%d.tif' % region_id

    dem = rasterio.open(dem_paths)
    dem_read = dem.read()
    dem_avg = np.mean(dem.read()[dem != -9999.0])


    for raster_path in raster_paths:
        raster_id = raster_path[raster_path.find('_2019')+1:raster_path.find('.tif')]
        raster = rasterio.open(raster_path)

        raster_arr = raster.read()
        min_shape_1 = min(raster_arr.shape[1],dem_read.shape[1])
        min_shape_2 = min(raster_arr.shape[2],dem_read.shape[2])
        x_pad = DIM-(min_shape_1%DIM)
        y_pad = DIM-(min_shape_2%DIM)

        dem_arr = np.pad(dem_read, ((0,0), (0, x_pad), (0, y_pad)), 'reflect')
        dem_arr[dem_arr == -9999.0] = dem_avg
        for x in range(0, min_shape_1-DIM+Step, DIM // Step):
            for y in range(0, min_shape_2-DIM+Step, DIM // Step):
                cropped_dem = dem_arr[:, x:x+DIM, y:y+DIM]
                cropped_raster = raster_arr[:, x:x+DIM, y:y+DIM]
                if len(cropped_dem) > 0 and cropped_dem.mean() > 800:
                    cropped_dem = ( cropped_dem - 28.320312 ) / (2468.8047 - 28.320312)
                    #sum_value = cropped_label.sum()
                    a1 = cropped_raster.astype('float32')/255.0
                    a2 = cropped_dem.astype('float32')
                    a3 = np.zeros((4,DIM,DIM))
                    a3[0:3] = a1
                    a3[3] = a2
                    tensor = torch.from_numpy(a3)
                    torch.save(tensor, output_dir + '/test_tiles/%d_%s_%d_%d.pt' % (region_id, raster_id, x, y))

                    csv_f.write(str(region_id) +  '\t' + raster_id + '\t' + str(x) +'\t' + str(y)  +'\n')
csv_f.close()

output direction: .
1
2
3
4
5
6


In [18]:
model_checkpoint = BASE_FOLDER + '/unet-epoch=243-dice_coeff=0.82.ckpt'

In [19]:
from __future__ import print_function, division
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset
import lightning.pytorch as pl
import torch.utils.data
from torch import optim


seed = 42


def set_seed(seed):
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)


set_seed(seed)

dataset_dir = "."

df = pd.read_csv(dataset_dir + '/test_tile_info.csv', sep='\t')


class TileDataset(Dataset):
    def __init__(
            self,
            folder,
            is_train=True
    ):
        super().__init__()
        self.folder = folder
        self.tile_info = pd.read_csv(folder + '/test_tile_info.csv', sep='\t')

    def __len__(self):
        return self.tile_info.shape[0]

    def __getitem__(self, index):
        region_id, raster_id, x, y = self.tile_info.iloc[index].values
        filename = '%d_%s_%d_%d.pt' % (region_id, raster_id, x, y)
        tile = torch.load(self.folder + '/test_tiles/' + filename)
        return tile.to(torch.float32)


torch.set_default_dtype(torch.float32)


class conv_block(nn.Module):
    """
    Convolution Block
    """

    def __init__(self, in_ch, out_ch, dropout=0):
        super(conv_block, self).__init__()

        self.conv = nn.Sequential(
            nn.Dropout(dropout),
            nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True))

    def forward(self, x):
        x = self.conv(x)
        return x


class up_conv(nn.Module):
    """
    Up Convolution Block
    """

    def __init__(self, in_ch, out_ch, dropout=0):
        super(up_conv, self).__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.up(x)
        return x


class U_Net(nn.Module):
    """
    UNet - Basic Implementation
    Paper : https://arxiv.org/abs/1505.04597
    """

    def __init__(self, in_ch=3, out_ch=1):
        super(U_Net, self).__init__()

        n1 = 64
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]

        self.Maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        # self.Maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        # self.Maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = conv_block(in_ch, filters[0], 0)
        self.Conv2 = conv_block(filters[0], filters[1], 0.1)
        self.Conv3 = conv_block(filters[1], filters[2], 0.1)
        # self.Conv4 = conv_block(filters[2], filters[3], 0.1)
        # self.Conv5 = conv_block(filters[3], filters[4], 0.1)

        # self.Up5 = up_conv(filters[4], filters[3])
        # self.Up_conv5 = conv_block(filters[4], filters[3], 0.1)

        # self.Up4 = up_conv(filters[3], filters[2])
        # self.Up_conv4 = conv_block(filters[3], filters[2], 0.1)

        self.Up3 = up_conv(filters[2], filters[1])
        self.Up_conv3 = conv_block(filters[2], filters[1], 0.1)

        self.Up2 = up_conv(filters[1], filters[0])
        self.Up_conv2 = conv_block(filters[1], filters[0], 0.1)

        self.Conv = nn.Conv2d(filters[0], out_ch, kernel_size=1, stride=1, padding=0)

        self.active = torch.nn.Sigmoid()

    def forward(self, x):
        e1 = self.Conv1(x)

        e2 = self.Maxpool1(e1)
        e2 = self.Conv2(e2)

        e3 = self.Maxpool2(e2)
        e3 = self.Conv3(e3)

        # e4 = self.Maxpool3(e3)
        # e4 = self.Conv4(e4)

        # e5 = self.Maxpool4(e4)
        # e5 = self.Conv5(e5)

        # d5 = self.Up5(e5)
        # d5 = torch.cat((e4, d5), dim=1)

        # d5 = self.Up_conv5(d5)

        # d4 = self.Up4(d5)
        # d4 = torch.cat((e3, d4), dim=1)
        # d4 = self.Up_conv4(d4)

        d3 = self.Up3(e3)  # d4
        d3 = torch.cat((e2, d3), dim=1)
        d3 = self.Up_conv3(d3)

        d2 = self.Up2(d3)
        d2 = torch.cat((e1, d2), dim=1)
        d2 = self.Up_conv2(d2)

        d1 = self.Conv(d2)

        out = self.active(d1)

        return out


class LitU_net(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.unet = U_Net(4, 1)

    def forward(self, x):
        return self.unet(x)

    def training_step(self, batch):
        x, y = batch
        y_pred = self.forward(x)
        loss = self.dice_loss(y_pred, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch):
        x, y = batch
        y_pred = self.forward(x)
        loss = self.dice_loss(y_pred, y)
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log("dice_coeff", self.dice_coeff(y_pred, y), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return y_pred

    def predict_step(self, batch):
        # x, y = batch
        y_pred = self.forward(batch)
        return y_pred

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def dice_loss(self, logits, targets):
        intersection = 2 * (logits * targets).sum() + 1  # self.smooth
        union = (logits + targets).sum() + 1  # self.smooth
        dice_loss = 1. - intersection / union

        loss = nn.BCELoss()
        bce_loss = loss(logits, targets)
        return 0.5 * dice_loss + 0.5 * bce_loss

    def dice_coeff(self, logits, targets):
        intersection = 2 * (logits * targets).sum()
        union = (logits + targets).sum()
        if union == 0:
            return 1
        dice_coeff = intersection / union
        return dice_coeff.item()


batch_size = 8
num_workers = 12

test_loader = torch.utils.data.DataLoader(TileDataset(dataset_dir), batch_size=batch_size, num_workers=num_workers)

model = LitU_net.load_from_checkpoint(model_checkpoint)
model.eval()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


LitU_net(
  (unet): U_Net(
    (Maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv1): conv_block(
      (conv): Sequential(
        (0): Dropout(p=0, inplace=False)
        (1): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU(inplace=True)
        (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): ReLU(inplace=True)
      )
    )
    (Conv2): conv_block(
      (conv): Sequential(
        (0): Dropout(p=0.1, inplace=False)
        (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3

In [20]:
trainer = pl.Trainer(accelerator="gpu")
predictions = trainer.predict(model, test_loader)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, low

Predicting: |          | 0/? [00:00<?, ?it/s]

In [21]:
import rasterio
import fiona
import rasterio.features
from rasterio.crs import CRS
from rasterio.features import rasterize
import shapely
from shapely import wkt
from shapely import envelope
from shapely.geometry import shape, mapping
from shapely.geometry.polygon import Polygon, LineString
import geopandas as gpd
from shapely.geometry.multipoint import MultiPoint
from shapely import buffer

In [22]:
output_dir = BASE_FOLDER+ '/predictions/'

shp_schema = {
    'geometry': 'Polygon',
    'properties': {
    'image': 'str',
    'region_num': 'int'}
}

In [23]:
def get_area(polygon, crs):
    return gpd.GeoDataFrame({'geometry': [polygon]}, crs=crs).area[0]
def process_polygon(polygon, crs):
    max_diagonal = -1
    area = get_area(polygon, crs)
    area2 = area
    if area >= 100000:
        bounding_rect = MultiPoint(polygon.exterior.coords).minimum_rotated_rectangle
        coords = bounding_rect.exterior.coords
        for i in range(len(coords)-1):
            for j in range(i+1, len(coords)-1):
                line = LineString([coords[i], coords[j]])
                max_diagonal = max(max_diagonal, line.length)
        # this was implemented based on this discussion: https://gis.stackexchange.com/questions/316128/identifying-long-and-narrow-polygons-in-with-postgis
        area2 = get_area(buffer(polygon, -0.085*max_diagonal), crs)
    return area, area2


In [25]:

import rasterio
tile_info = pd.read_csv('./test_tile_info.csv', sep='\t')
DIM=256
crs = 'EPSG:3857'
all_shapes = {i: {} for i in range(1,7)}

for region_id in range(1,7):
  raster_ids = np.unique(tile_info[tile_info['region_id'] == region_id]['raster_id'].values)
  print(raster_ids)
  for raster_id in raster_ids:
    raster = rasterio.open('/content/drive/MyDrive/Research/SIGSPATIAL Competition/data/test/region_0%d/Greenland26X_22W_Sentinel2_%s.tif' % (region_id, raster_id))
    raster_arr =raster.read()
    x_pad = DIM-(raster_arr.shape[1]%DIM)
    y_pad = DIM-(raster_arr.shape[2]%DIM)
    merged_labels = np.zeros((1, raster_arr.shape[1]+x_pad, raster_arr.shape[2]+y_pad),dtype=np.int32)
    tile_index = 0
    mask = (tile_info['region_id'] == region_id) & (tile_info['raster_id'] == raster_id)
    print(region_id, raster_id, mask.sum())
    for i in range(len(predictions)):
      for j in range(predictions[i].shape[0]):
        if mask[tile_index]:
          tile_pred = predictions[i][j].numpy()
          region_id, raster_id, x, y = tile_info.iloc[tile_index]
          tile_pred[tile_pred < 0.1] = 0
          tile_pred[tile_pred >= 0.1] = 1

          merged_labels[0, x:x+DIM, y:y+DIM] = tile_pred
        tile_index+=1

    with rasterio.open(
        BASE_FOLDER + '/predictions/%d_%s.tif' % (region_id, raster_id), "w",
        driver = "GTiff",
        crs = raster.crs,
        transform = raster.transform,
        dtype = rasterio.uint8,
        count = 1,
        width = raster.width,
        height = raster.height) as dst:
        dst.write(merged_labels[0,0:raster_arr.shape[1], 0:raster_arr.shape[2]], indexes = 1)
    mask = merged_labels == 1

    detected_shapes = rasterio.features.shapes(merged_labels, connectivity=8, transform=raster.transform)
    shapes = []
    for geom, value in detected_shapes:
        if value > 0:
            polygon = Polygon(shape(geom)).convex_hull
            area, area2 = process_polygon(polygon, crs)
            if area < 10000:
                continue
            if area2 ==  0:
                print("skipping narrow shape")
                continue
            shapes.append(polygon)

    gdf = gpd.GeoDataFrame({'geometry': shapes}, crs=crs)
    gdf['geometry'] = gdf['geometry'].buffer(120)
    gdf = gdf.dissolve().explode(index_parts=True)
    all_shapes[region_id][raster_id] = gdf

['2019-06-03_05' '2019-07-31_25']
1 2019-06-03_05 1560
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
1 2019-07-31_25 1560
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
['2019-06-19_20' '2019-08-25_29']
2 2019-06-19_20 1494
skipping narrow shape
skipping narrow shape
skipping narrow shape
2 2019-08-25_29 1494
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
skipping narrow shape
['2019-06-03_05' '2019-07-31_25']
3 2019-06-03_05 1700
3 2019-07-31_25 1700
skipping narrow shape
skipping narrow shape
skipping narrow shape
sk

In [26]:
with fiona.open(BASE_FOLDER+'/lake_polygons_test.gpkg', 'w', 'GPKG',shp_schema, crs) as shp:
  for region_id in all_shapes:
    for raster_id in all_shapes[region_id]:
      shapes = all_shapes[region_id][raster_id]['geometry'].values
      image = 'Greenland26X_22W_Sentinel2_%s.tif' % raster_id
      print(region_id, image)
      for polygon in shapes:
        polygon = polygon.convex_hull
        shp.write({
            'geometry': mapping(polygon),
            'properties': {
            'image':image,
            'region_num': int(region_id)
            }
        })


1 Greenland26X_22W_Sentinel2_2019-06-03_05.tif
1 Greenland26X_22W_Sentinel2_2019-07-31_25.tif
2 Greenland26X_22W_Sentinel2_2019-06-19_20.tif
2 Greenland26X_22W_Sentinel2_2019-08-25_29.tif
3 Greenland26X_22W_Sentinel2_2019-06-03_05.tif
3 Greenland26X_22W_Sentinel2_2019-07-31_25.tif
4 Greenland26X_22W_Sentinel2_2019-06-19_20.tif
4 Greenland26X_22W_Sentinel2_2019-08-25_29.tif
5 Greenland26X_22W_Sentinel2_2019-06-03_05.tif
5 Greenland26X_22W_Sentinel2_2019-07-31_25.tif
6 Greenland26X_22W_Sentinel2_2019-06-19_20.tif
6 Greenland26X_22W_Sentinel2_2019-08-25_29.tif
